In [160]:
import numpy as np
fname = 'input_day15.txt'

In [26]:
data = []
with open(fname) as f:
    while True:
        d = list(f.readline()[:-1])
        if len(d)>1:
            data.append(d)
        else:
            break
data = np.array(data)

In [353]:
dudes = np.where((data.flatten() == 'G') | (data.flatten() == 'E'))[0]
is_elf = data.flatten()[dudes] == 'E'
hp = np.array([200 for ele in dudes])

In [36]:
is_elf = data.flatten()[dudes] == 'E'

In [38]:
dudes

array([ 85,  89, 132, 148, 165, 216, 329, 342, 343, 373, 377, 405, 418,
       458, 459, 485, 491, 551, 585, 618, 628, 650, 681, 682, 690, 701,
       712, 783, 915, 979])

In [44]:
data.shape

(32, 32)

In [122]:
def dup_rows(a):
    for indx in range(a.shape[0])[::-1]:
        a = np.insert(a,[indx+1]*3,a[indx],axis=0)
    return a

In [513]:
board = data.copy()
dudes = np.where((board.flatten() == 'G') | (board.flatten() == 'E'))[0]
is_elf = board.flatten()[dudes] == 'E'
hp = np.array([200 for ele in dudes])
steps = (np.array([1,0]), np.array([-1,0]), np.array([0,1]), np.array([0,-1]))
nround = 0
while True:
    #print('asdf: {}'.format(len(dudes)))
    ordering = np.argsort(dudes)
    dudes = dudes[ordering] #locations (1-d) of dudes
    is_elf = is_elf[ordering] #boolean
    hp = hp[ordering] #int, starts at 200
    
    unrav_loc = np.array(np.unravel_index(dudes, board.shape, order='C')).T
    asdf = False
    i=0
    while i < len(dudes):
        target = None
        bad_guy = 'G' if is_elf[i] else 'E'
        good_guy = 'E' if is_elf[i] else 'G'
        
        currentloc = unrav_loc[i:i+1,:] #the i:i+1 makes it keep the 2nd dimension
        adj_squares = np.tile(currentloc, (4,1)) + steps
        
        can_attack = False
        
        if bad_guy in board[tuple(adj_squares.T)]:
            can_attack = True
        else:
            searchmap = board.copy().astype('U2')
            stepcount = 1
            cloc = currentloc.copy()
            searchmap[tuple(currentloc.T)] = 'X'
            cant_move = False
            while True:
                search_adj = dup_rows(cloc) + np.tile(steps, (cloc.shape[0],1))
                is_bad_in_search = searchmap[tuple(search_adj.T)] == bad_guy
                adj_baddies = search_adj[is_bad_in_search]
                if len(adj_baddies) > 0:
                    #print('found bad guy {} steps away'.format(stepcount))
                    if len(adj_baddies) > 1:
                        spot = np.sort([ele[0]*board.shape[0] + ele[1] for ele in adj_baddies])[0]
                        target = np.array(np.unravel_index(spot, board.shape, order='C')).T

                    else:
                        target = adj_baddies[0]
                    break
                search_adj = np.unique(search_adj, axis=0)
                open_adj = search_adj[searchmap[tuple(search_adj.T)] == '.',:]
                if len(open_adj) == 0:
                    cant_move = True
                    break
                searchmap[tuple(open_adj.T)] = str(stepcount)
                cloc = open_adj
                stepcount +=1
                asdf = True
                
            if cant_move:
                i+=1
                continue
            
            cloc = target
            stepcount -=1
            while True:
                open_adj = np.where(searchmap == str(stepcount))
                if cloc.ndim == 1:
                    cloc = np.expand_dims(cloc, axis=0)
                search_adj = dup_rows(cloc) + np.tile(steps, (cloc.shape[0],1))
                cloc = np.array([ele for ele in search_adj.tolist() if ele in np.array(open_adj).T.tolist()])
                cloc = np.unique(cloc,axis=0)
                stepcount -=1
                if stepcount == 0:
                    spot = np.sort([ele[0]*board.shape[0] + ele[1] for ele in cloc])[0]
                    next_step = np.array(np.unravel_index(spot, board.shape, order='C')).T
                    board[tuple(next_step.T)] = good_guy
                    board[tuple(currentloc.T)] = '.'
                    unrav_loc[i] = next_step.T
                    break
                
            adj_squares = np.tile(next_step, (4,1)) + steps
            
        
        if bad_guy in board[tuple(adj_squares.T)]:
            #print('we are going to attack!')
            baddies_near = board[tuple(adj_squares.T)] == bad_guy
            squarenums_with_baddies = adj_squares[baddies_near,0]*board.shape[0] + adj_squares[baddies_near,1]
            baddies_ind = np.array([np.where(ele == dudes)[0][0] for ele in squarenums_with_baddies])
            hp_of_baddies = hp[baddies_ind]
            chosen_baddie = np.argsort(1e6*(200 - hp_of_baddies) + baddies_ind)[0]
            #print(chosen_baddie)
            target_ind = baddies_ind[chosen_baddie]
            target = np.unravel_index(squarenums_with_baddies[chosen_baddie], board.shape, order='C')
            can_attack = True
        else:
            can_attack = False
                    
        #COMBAT
        if can_attack:
            badguy_index = target_ind
            #print('attacking {}, loc {}, hp {}'.format(badguy_index, target, hp[badguy_index]))
            hp[badguy_index] = hp[badguy_index] - 3
            if hp[badguy_index] < 1:
                print(target)
                board[target] = '.'
                
        #remove dead dudes
        dudes = np.array([ele[0]*board.shape[0] + ele[1] for ele in unrav_loc])
        dudes = dudes[hp>0]
        is_elf = is_elf[hp>0]
        unrav_loc = unrav_loc[hp>0, :]
        hp = hp[hp>0]
            
        i+=1

    #print(hp[is_elf])
    #print(hp[~is_elf])
    #print("\n".join(["".join(ele) for ele in board])); print('\n')
    if all(is_elf[hp>0]) or all(~is_elf[hp>0]):
        print('no enemies remaining!')
        print('outcome: {}'.format((nround)*sum(hp[hp>0])))
        break
        
    nround +=1


(10, 23)
(20, 10)
(11, 21)
(15, 11)
(13, 11)
(20, 19)
(23, 11)
(20, 12)
(21, 11)
(20, 20)
(21, 20)
no enemies remaining!
outcome: 190450


In [498]:
board[target]

array(['E'], dtype='<U1')

In [458]:
np.argsort(range(5))

array([0, 1, 2, 3, 4])

In [440]:
nround

69

In [441]:
69*2981

205689

In [372]:
board[tuple(adj_squares.T)]

array(['.', 'G', '.', '.'], dtype='<U1')

In [432]:
cloc.shape

(17400, 2)

In [335]:
unrav_loc

array([[ 2, 21],
       [ 2, 25],
       [ 4,  4],
       [29, 19],
       [ 5,  5],
       [ 6, 24],
       [10,  9],
       [10, 22],
       [10, 23],
       [11, 21],
       [11, 25],
       [12, 21],
       [13,  2],
       [14, 10],
       [14, 11],
       [15,  5],
       [15, 11],
       [17,  7],
       [18,  9],
       [19, 10],
       [19, 20],
       [20, 10],
       [21,  9],
       [21, 10],
       [21, 18],
       [21, 29],
       [22,  8],
       [24, 15],
       [28, 19],
       [30, 19]])

In [507]:
print("\n".join(["".join(ele) for ele in board]))

################################
###################........#####
###################.....#...####
####################........####
##...###############.......#####
###...###############.....######
#####.######..######.....##..###
#####.........####............##
#########...#####.............##
#########...####.......G......##
#########.#####......GGEG.....##
#########...........GEGG.....###
######.###....#####..G........##
#..#....##...#######.........###
##.#....##.G#########.........##
#.....#...GE#########....#....##
#..........G#########.......####
#####.......#########...##....##
#####.......#########.#.......##
#######.G.G..#######......#...##
######...GEG..#####............#
######....G..............#.....#
#######....G#....#..#...#.....##
#######...G.G....#####..####.###
########...G....################
#######..........###############
########.............###########
#########...#...##....##########
#########.....#.#.....##########
################.....###########
##########

In [260]:
search_adj.tolist()

[[12, 21], [10, 21], [11, 22], [11, 20]]

In [261]:
[ele for ele in search_adj.tolist() if ele in np.array(open_adj).T.tolist()]

[[10, 21]]

In [263]:
np.array([[2,1],[2,2],[1,1],[1,2]])

array([[2, 1],
       [2, 2],
       [1, 1],
       [1, 2]])

In [264]:
np.sort(np.array([[2,1],[2,2],[1,1],[1,2]]),axis=0)

array([[1, 1],
       [1, 1],
       [2, 2],
       [2, 2]])

In [257]:
list(np.array(open_adj).T)

[array([ 2, 27]),
 array([ 7, 18]),
 array([ 7, 24]),
 array([ 8, 19]),
 array([ 8, 23]),
 array([ 9, 20]),
 array([ 9, 22]),
 array([10, 21])]

In [250]:
np.array(open_adj).T

array([[ 2, 27],
       [ 7, 18],
       [ 7, 24],
       [ 8, 19],
       [ 8, 23],
       [ 9, 20],
       [ 9, 22],
       [10, 21]])

In [243]:
searchmap == '8'

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [225]:
np.unique(open_adj,axis=0)

array([[ 1, 20],
       [ 1, 22],
       [ 2, 19],
       [ 2, 23],
       [ 3, 20],
       [ 3, 22],
       [ 4, 21]])

In [430]:
stepcount

3

In [429]:
print("\n".join(["".join(ele) for ele in board]))

################################
###################........#####
###################.....#...####
####################........####
##...###############.......#####
###...###############.....######
#####.######..######.....##..###
#####.........####............##
#########...#####.............##
#########...####G.............##
#########.#####..G............##
#########......G.............###
######.###.G..#####...........##
#..#....##GE.#######.....G...###
##.#....##.G#########.........##
#.....#...GE#########....#....##
#..........G#########.G.....####
#####.......#########G.G##....##
#####.......#########.#.......##
#######.....G#######......#...##
######.....GE.#####GE..........#
######....GE.......E.....#.....#
#######....G#....#..#...#.....##
#######...GE.....#####..####.###
########...GG...################
#######..........###############
########.............###########
#########...#...##....##########
#########.....#.#.....##########
################.....###########
##########

In [428]:
print("\n".join(["".join(ele) for ele in searchmap]))

################################
###################1514131415161718#####
###################1413121314#161718####
####################1211121314151617####
##...###############11101112131415#####
###...###############910111213######
#####.######..######7891011##1415###
#####.........####456789101112131415##
#########...#####234567891011121314##
#########...####X12345678910111213##
#########.#####21G34567891011121314##
#########......G234567891011121314###
######.###.G..#####678910111213141516##
#..#....##GE.#######89101112G141516###
##.#....##.G#########101112131415161718##
#.....#...GE#########11121314#16171819##
#..........G#########12G1415161718####
#####.......#########G.G##18192021##
#####.......#########26#22212019202122##
#######.....G#######262524232221#212223##
######.....GE.#####GE26252423222322232425#
######....GE.......E..262524#2423242526#
#######....G#....#..#..26#2625242526##
#######...GE.....#####..####26###
########...GG...################
#######..........############

In [75]:
board = data.copy()
unrav_loc = np.array(np.unravel_index(dudes, board.shape, order='C')).T
steps = ((1,0), (-1,0), (0,1), (0,-1))

In [48]:
steps = (np.array([1,0]), np.array([-1,0]), np.array([0,1]), np.array([0,-1]))

In [144]:
is_bad_in_search

array([False, False, False, False])

In [139]:
searchmap[search_adj[:,0], search_adj[:,1]]

array(['.', '#', '#', '.'], dtype='<U1')

In [138]:
search_adj.shape

(4, 2)

In [149]:
searchmap[tuple(search_adj.T)]

array(['.', '#', '#', '.'], dtype='<U1')

In [152]:
searchmap[tuple(search_adj.T)] == '.'

array([ True, False, False,  True])

In [154]:
search_adj[searchmap[tuple(search_adj.T)] == '.']

array([[5, 4],
       [4, 3]])

In [157]:
searchmap[tuple(search_adj.T)]

array(['.', '#', '#', '.'], dtype='<U1')